In [34]:
from sympy import init_session
import math
import matplotlib.pyplot as plt
import sys
import ipywidgets as widgets
import numpy as np
import imageio.v2 as imageio
from IPython.display import Image
import pandas as pd
import seaborn as sns

In [78]:
# Size

a = 1.97 * 25.4 #mm R2
b = 12.4 * 25.4 #mm R3
c = 3.17 * 25.4 #mm R4
d = 13.4 * 25.4 #mm R1

a, b, c, d

(50.038, 314.96, 80.518, 340.36)

In [35]:
m = 3.48 #kg
l = 24.5 * 25.4 #length of wiper mm
I = 1/12 * m * l**2 #moment of inertia about com kgm^2

In [132]:
#Parameters
motor_torque = 20 #nM 
df = pd.read_csv("4bardata.txt", sep='\t' , header = None, names = ["theta_2 [rad]", "theta_3 [rad]", "theta_4 [rad]", "theta_2 [°]","theta_3 [°]","theta_4 [°]","omega_2 [rad/s]", "omega_43 [rad/s]","omega_4 [rad/s]","alpha_2 [rad/s^2]","alpha_3 [rad/s^2]","alpha_4 [rad/s^2]"])
#df.head()
#print(df.shape[0])

for i in range(df.shape[0]):
    print(df['theta_2 [rad]'][i])

4.71238898038469
9.375288980384688
14.08528898038469
18.79528898038469


KeyError: 4

In [101]:
#Torque Calculations

for i in range(df.shape[0]):
    tau1 = []
    if np.pi/2 >= df["theta_2 [rad]"][i] >0:
        temp1 = a * motor_torque * np.sin(df["theta_2 [rad]"][i]) 
    elif np.pi >= df["theta_2 [rad]"][i] > np.pi/2:
        temp1 = a * motor_torque * np.sin(np.pi - df["theta_2 [rad]"][i])
    elif 3 * np.pi/2 >= df["theta_2 [rad]"][i] > np.pi:
        temp1 = a * motor_torque * np.sin(df["theta_2 [rad]"][i] - np.pi)
    elif (2* np.pi) >= df["theta_2 [rad]"][i] > (3 * np.pi/2) :
        temp1 = a* motor_torque * np.sin(2 * np.pi - df["theta_2 [rad]"][i])
    
    tau1.append(temp1)


KeyError: 4